In [1]:
#!/usr/bin/python3.3
# -*-coding:Utf-8 -*


import pandas as pd 
import statsmodels.formula.api as smf
import statsmodels.api as sm
import matplotlib.pyplot as plt 
from datetime import datetime
pd.options.mode.chained_assignment = None  # default='warn'
import xgboost as xgb
# reading data
train = pd.read_csv("sources/train.csv")
test = pd.read_csv("sources/test.csv")

In [2]:
#add a new column 'date' convert into datetime for both training and test set
train['date']=0
for i in range(len(train)) :
	train['date'][i]=datetime.strptime(train['datetime'][i],'%Y-%m-%d %H:%M:%S')

train=train.set_index("datetime")


test['date']=0
for i in range(len(test)) :
	test['date'][i]=datetime.strptime(test['datetime'][i],'%Y-%m-%d %H:%M:%S')

test=test.set_index("datetime")

In [3]:
#create day of week column and hour column and a year colum for both training and test set
train['day']=0
train['hour']=0
train['year']=0
for i in range(len(train)) :
    train['day'][i] = train['date'][i].weekday()
    train['hour'][i] = train['date'][i].hour
    train['year'][i]=train['date'][i].year

test['day']=0
test['hour']=0
test['year']=0
for i in range(len(test)) :
    test['day'][i] = test['date'][i].weekday()
    test['hour'][i] = test['date'][i].hour
    test['year'][i]=test['date'][i].year


In [4]:
#create weekEnd variable 
train['weekEnd'] = 0 ;
train.ix[train['day']>=5,'weekEnd']=1

test['weekEnd'] = 0 ;
test.ix[test['day']>=5,'weekEnd']=1


In [5]:
#create ahot () variable for both training and test set
train['ahot'] = 4
for i in range(len(train)) :
	currentTemp = train['atemp'][i]
	if (currentTemp>=24.24 and currentTemp<31.06 ):
		train['ahot'][i] = 3
	elif (currentTemp>=16.6 and currentTemp<24.24):
		train['ahot'][i] = 2
	elif ( currentTemp<16.6) :
		train['ahot'][i] = 1


test['ahot'] = 4
for i in range(len(test)) :
	currentTemp = test['atemp'][i]
	if (currentTemp>=24.24 and currentTemp<31.06 ):
		test['ahot'][i] = 3
	elif (currentTemp>=16.6 and currentTemp<24.24):
		test['ahot'][i] = 2
	elif (currentTemp<16.6) :
		test['ahot'][i] = 1



In [6]:
#create ahot () variable for both training and test set
train['hot'] = 4
for i in range(len(train)) :
	currentTemp = train['temp'][i]
	if (currentTemp>=20.5 and currentTemp<26.24 ):
		train['hot'][i] = 3
	elif (currentTemp>=13.94 and currentTemp<20.5):
		train['hot'][i] = 2
	elif ( currentTemp<13.94) :
		train['hot'][i] = 1


test['hot'] = 4
for i in range(len(test)) :
	currentTemp = test['temp'][i]
	if (currentTemp>=20.5 and currentTemp<26.24 ):
		test['hot'][i] = 3
	elif (currentTemp>=13.94 and currentTemp<20.5):
		test['hot'][i] = 2
	elif (currentTemp<13.94) :
		test['hot'][i] = 1



In [7]:
train = train.reset_index()
train = train.drop("datetime",1)
train = train.drop(['date'],1)

test = test.reset_index()
result= test[['datetime']]

test = test.drop("datetime",1)
test = test.drop('date',1)

In [8]:
train_registered=train.drop("casual",1)
train_registered=train_registered.drop("count",1)

train_casual=train.drop("registered",1)
train_casual=train_casual.drop("count",1)

In [9]:
train_registered.columns

Index(['season', 'holiday', 'workingday', 'weather', 'temp', 'atemp',
       'humidity', 'windspeed', 'registered', 'day', 'hour', 'year', 'weekEnd',
       'ahot', 'hot'],
      dtype='object')

In [10]:
X_train = train_registered.drop("registered",1)
Y_train = train_registered['registered']
T_train_xgb = xgb.DMatrix(X_train, Y_train)
params = {}
params["objective"] = "count:poisson"
params["bst:eta"] = 1
#params["min_child_weight"] = 5
#params["subsample"] = 0.8
params["scale_pos_weight"] = 1.0
params["silent"] = 1
params["max_depth"] = 9
params["max_delta_step"]=1
gbm_reg = xgb.train(dtrain=T_train_xgb, params = params)
X_test = xgb.DMatrix(test)
Y_pred_reg = gbm_reg.predict(X_test)
print(Y_pred_reg)
xgb.plot_importance(gbm_reg)


[  12.70296955    4.45095348    2.87567091 ...,  116.40880585  100.00267792
   47.491642  ]


In [11]:
plt.show()

In [12]:
X_train = train_casual.drop("casual",1)
Y_train = train_casual['casual']
T_train_xgb = xgb.DMatrix(X_train, Y_train)
params = {}
params["objective"] = "count:poisson"
params["bst:eta"] = 1
#params["min_child_weight"] = 5
#params["subsample"] = 0.8
params["scale_pos_weight"] = 1.0
params["silent"] = 1
params["max_depth"] = 10
params["max_delta_step"]=1
gbm_cas = xgb.train(dtrain=T_train_xgb, params = params)
X_test = xgb.DMatrix(test)
Y_pred_cas = gbm_cas.predict(X_test)
print(Y_pred_cas)
xgb.plot_importance(gbm_cas)


[ 2.10598421  0.65608001  0.59178776 ...,  3.77964616  2.38016868
  1.68373263]


In [13]:
plt.show()

In [14]:
result['count'] = pd.DataFrame( Y_pred_cas + Y_pred_reg)
#result[result['count']<=0] = 0

result_final = result.set_index('datetime')
result_final[result_final['count']<=0]=0


In [15]:
result_final.to_csv('Résultats/XGBoost/submissionXGBoost.csv')